In [11]:
# Import Pandas
import pandas as pd
import numpy as np

# Load Movies dataset
df = pd.read_csv('C:\\Users\\kaoutharben\\Documents\\flask_app\\dataSet\\netflix_titles.csv')
df.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [12]:
list(df.columns)

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

In [4]:
#select of variables that can affect effictively system result 
features = ['description','type','director','cast','country','rating','listed_in']

#filling all NaNs
for feature in features:
    df[feature] = df[feature].fillna('')


#function that combines chosen features into a single string
def merge_columns(movie):
    return movie['description']+" "+movie['type']+" "+movie['cast']+" "+movie['director']+movie['country']+movie['rating']+movie['listed_in']


NameError: name 'df' is not defined

In [109]:

df["combined_features"] = df.apply(merge_columns,axis=1)
df["combined_features"][0]

'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor. TV Show João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi BrazilTV-MAInternational TV Shows, TV Dramas, TV Sci-Fi & Fantasy'

In [110]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define TF-IDF vector
vector = TfidfVectorizer(   max_df=0.4,         # drop words that occur in more than X percent of documents
                            min_df=1,      # only use words that appear at least X times
                            stop_words='english', # remove stop words
                            lowercase=True, # Convert everything to lower case 
                            use_idf=True,   # Use idf
                            norm=u'l2',     # Normalization
                            smooth_idf=True # Prevents divide-by-zero errors
                            )

#Define TF-IDF matrix where each column represents a word, and each row represents a movie
tfidf_matrix = vector.fit_transform(df["combined_features"])



In [111]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [112]:
#calculate the scalar products of multiple vectors
def scalar_product(vectors):
  product=vectors[0]
  i=1
  while i<vectors.get_shape()[0]:
    product=np.dot(product, vectors[i])
    i+=1
  return product

In [129]:
# Import linear_kernel (faster than cosine_similarities)
from sklearn.metrics.pairwise import linear_kernel


def recommendations(titles):

  if (len(titles)==0):
    print("Please select your favorite movie")
    return

  elif (len(titles)==1):
    movie_index = indices[titles[0]]
    # Compute the cosine similarity matrix
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)   
 #list of similar movies in the form of(movie_index, similarity_rate)
    similar_movies = list(enumerate(cosine_similarities[movie_index]))
  #sorting similar movies in descending order with eliminating the first element 
 #wich is the movie itself
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:6]
  
    
   
 
  else :

    product_vect=tfidf_matrix[indices[titles[0]]]
    indices_slected_titles=[]

    for e in titles:
       indices_slected_titles.append(indices[e])
       matrix_result = (product_vect.tocsr() +tfidf_matrix[indices[e]].tocsr()).tolil()
    modif_tfidf_matrix=tfidf_matrix
    modif_tfidf_matrix[0]= scalar_product(product_vect )
    cosine_similarities = linear_kernel(modif_tfidf_matrix, tfidf_matrix)
    similar_movies = list(enumerate(cosine_similarities[0]))
     #sorting similar movies in descending order with eliminating the indexes of chosen movies
    check =True
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[:8]
    list_repeated_item=[]
    for e in sorted_similar_movies:
     
      for item in indices_slected_titles:
        if (e[0]==item):
          check=False
          list_repeated_item.append(e)
       
     
    
    if check==False:
      if len( list_repeated_item)==1:
         sorted_similar_movies.remove(list_repeated_item[0])
      if len( list_repeated_item)==2:
        for e in list_repeated_item:
         sorted_similar_movies.remove(e)
         
    
# Get the movie indices
  movie_indices = [i[0] for i in sorted_similar_movies[:5]]
  print("Recommendation list is  : \n")
  return df['title'].iloc[movie_indices]


In [130]:
recommendations(["Ocean's Twelve","Stranger Things"])

[4565, 5836]
Recommendation list is  : 



0                                    3%
4564                   Ocean's Thirteen
6726       The Original Kings of Comedy
1281    Charlie's Angels: Full Throttle
6491                     The Informant!
Name: title, dtype: object